In [1]:
import sys
sys.path.insert(1, '/trinity/home/xwan/MPNST_DL/working')
from dataset import MPNSTDataMoule, MPNSTDataset
from networks import spp
from networks.resnet3d_spp import generate_model
from torchsummary import summary
import torch

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


/trinity/home/xwan/MPNST_DL/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = generate_model(10)

In [11]:
a = model(torch.rand(1, 512, 256, 80).unsqueeze_(0))
b = model(torch.rand(1, 512, 512, 80).unsqueeze_(0))


In [13]:
print(a.shape)
print(b.shape)

torch.Size([1, 2])
torch.Size([1, 2])


In [9]:
summary(model, torch.rand(1, 512, 256, 80).unsqueeze_(0))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv3d: 1-1                            [-1, 64, 512, 128, 40]    21,952
├─BatchNorm3d: 1-2                       [-1, 64, 512, 128, 40]    128
├─ReLU: 1-3                              [-1, 64, 512, 128, 40]    --
├─Sequential: 1-4                        [-1, 64, 512, 128, 40]    --
|    └─BasicBlock: 2-1                   [-1, 64, 512, 128, 40]    --
|    |    └─Conv3d: 3-1                  [-1, 64, 512, 128, 40]    110,592
|    |    └─BatchNorm3d: 3-2             [-1, 64, 512, 128, 40]    128
|    |    └─ReLU: 3-3                    [-1, 64, 512, 128, 40]    --
|    |    └─Conv3d: 3-4                  [-1, 64, 512, 128, 40]    110,592
|    |    └─BatchNorm3d: 3-5             [-1, 64, 512, 128, 40]    128
|    |    └─ReLU: 3-6                    [-1, 64, 512, 128, 40]    --
├─Sequential: 1-5                        [-1, 128, 256, 64, 20]    --
|    └─BasicBlock: 2-2                   [-1, 128, 256, 64, 20]    -

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv3d: 1-1                            [-1, 64, 512, 128, 40]    21,952
├─BatchNorm3d: 1-2                       [-1, 64, 512, 128, 40]    128
├─ReLU: 1-3                              [-1, 64, 512, 128, 40]    --
├─Sequential: 1-4                        [-1, 64, 512, 128, 40]    --
|    └─BasicBlock: 2-1                   [-1, 64, 512, 128, 40]    --
|    |    └─Conv3d: 3-1                  [-1, 64, 512, 128, 40]    110,592
|    |    └─BatchNorm3d: 3-2             [-1, 64, 512, 128, 40]    128
|    |    └─ReLU: 3-3                    [-1, 64, 512, 128, 40]    --
|    |    └─Conv3d: 3-4                  [-1, 64, 512, 128, 40]    110,592
|    |    └─BatchNorm3d: 3-5             [-1, 64, 512, 128, 40]    128
|    |    └─ReLU: 3-6                    [-1, 64, 512, 128, 40]    --
├─Sequential: 1-5                        [-1, 128, 256, 64, 20]    --
|    └─BasicBlock: 2-2                   [-1, 128, 256, 64, 20]    -

In [5]:
def get_inplanes():
    return [64, 128, 256, 512]


def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                    out_planes,
                    kernel_size=3,
                    stride=stride,
                    padding=1,
                    bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                    out_planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False)

In [6]:
from networks.resnet3d_spp import BasicBlock, Bottleneck
from networks.spp import SPP3DLayer

def _make_layer(block, planes, blocks, in_planes, stride=1):
        downsample = None
        if stride != 1 or in_planes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1x1(in_planes, planes * block.expansion, stride),
                nn.BatchNorm3d(planes * block.expansion)
            )
        layers = []
        layers.append(
            block(in_planes=in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        in_planes = planes * block.expansion
        for i in range (1, blocks):
            layers.append(block(in_planes, planes))
        
        return nn.Sequential(*layers)

In [7]:
block_inplanes = [int(x * 1.0) for x in get_inplanes()]
block_inplanes

[64, 128, 256, 512]

In [38]:
n_input_channels = 1
conv1_t_size = 7
conv1_t_stride = 2
scales = [3, 1]
layers = [1, 1, 1, 1]
n_classes = 2

In [39]:
in_planes = block_inplanes[0]
conv1 = nn.Conv3d(n_input_channels, 
                        in_planes,
                        kernel_size=(conv1_t_size, 7, 7),
                        stride=(conv1_t_stride, 2, 2),
                        padding=(conv1_t_size // 2, 3, 3),
                        bias=False)
bn1 = nn.BatchNorm3d(in_planes)
relu = nn.ReLU(inplace=True)
maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
block = BasicBlock
layer1 = _make_layer(BasicBlock, block_inplanes[0], layers[0], in_planes=in_planes)
in_planes = block_inplanes[0]

layer2 = _make_layer(BasicBlock, block_inplanes[1], layers[1], in_planes=in_planes, stride=2)
in_planes = block_inplanes[1]

layer3 = _make_layer(BasicBlock, block_inplanes[2], layers[2], in_planes=in_planes, stride=2)
in_planes = block_inplanes[2]

layer4 = _make_layer(BasicBlock, block_inplanes[3], layers[3], in_planes=in_planes, stride=2)
in_planes = block_inplanes[3]

spp = SPP3DLayer(scales)
linear = nn.Linear(block_inplanes[3]*sum([x**3 for x in scales]), n_classes)

In [40]:
test_data = torch.rand(1, 512, 512, 512)
test = test_data.unsqueeze_(0)

In [41]:
test
print(test.shape)

x = conv1(test)
print(x.shape)

x = bn1(x)
print(x.shape)

x = relu(x)
x = maxpool(x)
print(x.shape)

torch.Size([1, 1, 512, 512, 512])
torch.Size([1, 64, 256, 256, 256])
torch.Size([1, 64, 256, 256, 256])
torch.Size([1, 64, 128, 128, 128])


In [42]:
x = layer1(x)
print(x.shape)


torch.Size([1, 64, 128, 128, 128])


In [43]:
x = layer2(x)
print(x.shape)


torch.Size([1, 128, 64, 64, 64])


In [44]:
x = layer3(x)
print(x.shape)


torch.Size([1, 256, 32, 32, 32])


In [45]:
x = layer4(x)
print(x.shape)


torch.Size([1, 512, 16, 16, 16])


In [50]:
x.shape

torch.Size([1, 512, 16, 16, 16])

In [47]:
batch_size, in_channel, in_h, in_w, in_t = x.size()
scale_list = scales
for i in range(len(scale_list)):
    h_wid = int(math.ceil(in_h / scale_list[i]))
    w_wid = int(math.ceil(in_w / scale_list[i]))
    t_wid = int(math.ceil(in_t / scale_list[i]))
    h_pad = int((h_wid*scale_list[i] - in_h + 1)/2)
    w_pad = int((w_wid*scale_list[i] - in_w + 1)/2)
    t_pad = int((t_wid*scale_list[i] - in_t + 1)/2)
    maxpool = nn.MaxPool3d((h_wid, w_wid, t_wid), stride=(h_wid, w_wid, t_wid), padding=(h_pad, w_pad, t_pad))
    out = maxpool(x)
    print(out.shape)
    print(out.view(batch_size, -1).shape)
    if(i == 0):
        spp = out.view(batch_size, -1)
    else:
        spp = torch.cat((spp, out.view(batch_size, -1)), 1)

torch.Size([1, 512, 3, 3, 3])
torch.Size([1, 13824])
torch.Size([1, 512, 1, 1, 1])
torch.Size([1, 512])


In [51]:
spp.shape

torch.Size([1, 14336])

In [48]:
linear

Linear(in_features=14336, out_features=2, bias=True)

In [36]:
linear

Linear(in_features=14336, out_features=2, bias=True)

In [52]:
x_o = linear(spp)
x_o.shape

torch.Size([1, 2])

In [16]:
batch_size, in_channel, in_h, in_w, in_t = x.size()
print(x.size())

torch.Size([1, 512, 16, 16, 16])


In [96]:
scale_list = [3, 1]
i = 0

In [97]:
h_wid = int(math.ceil(in_h / scale_list[i]))
w_wid = int(math.ceil(in_w / scale_list[i]))
t_wid = int(math.ceil(in_t / scale_list[i]))
h_pad = int((h_wid*scale_list[i] - in_h + 1)/2)
w_pad = int((w_wid*scale_list[i] - in_w + 1)/2)
t_pad = int((t_wid*scale_list[i] - in_t + 1)/2)
maxpool = nn.MaxPool3d((h_wid, w_wid, t_wid), stride=(h_wid, w_wid, t_wid), padding=(h_pad, w_pad, t_pad), )
maxpool

MaxPool3d(kernel_size=(6, 6, 6), stride=(6, 6, 6), padding=(1, 1, 1), dilation=1, ceil_mode=False)

In [98]:
out = maxpool(x)
print(out.shape)

torch.Size([1, 512, 3, 3, 3])


In [99]:
spp = out.view(batch_size, -1)
print(spp.shape)

torch.Size([1, 13824])


In [100]:
i = 1
h_wid = int(math.ceil(in_h / scale_list[i]))
w_wid = int(math.ceil(in_w / scale_list[i]))
t_wid = int(math.ceil(in_t / scale_list[i]))
h_pad = int((h_wid*scale_list[i] - in_h + 1)/2)
w_pad = int((w_wid*scale_list[i] - in_w + 1)/2)
t_pad = int((t_wid*scale_list[i] - in_t + 1)/2)
maxpool_1 = nn.MaxPool3d((h_wid, w_wid, t_wid), stride=(h_wid, w_wid, t_wid), padding=(h_pad, w_pad, t_pad), )
maxpool_1

MaxPool3d(kernel_size=(17, 17, 17), stride=(17, 17, 17), padding=(0, 0, 0), dilation=1, ceil_mode=False)

In [101]:
out_1 = maxpool_1(x)
print(out_1.shape)

torch.Size([1, 512, 1, 1, 1])


In [102]:
spp = torch.cat((spp, out.view(batch_size, -1)), 1)

In [103]:
spp.shape

torch.Size([1, 27648])